In [4]:
!pip install torch==2.2.2 stable-baselines3==2.3.2 gymnasium==0.29.1 \
          yfinance==0.2.38 ta==0.10.2 --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
  

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os, json, math, typing, warnings, itertools, functools
from datetime import datetime
import numpy as np
import pandas as pd
import torch

import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise
import matplotlib.pyplot as plt
from tqdm import tqdm

In [37]:
# -----------------------------------------------------------------------
# 1. 경로 & 하이퍼파라미터
# -----------------------------------------------------------------------
DRIVE_PREFIX         = '/content/drive/MyDrive/객지프'
DATA_DIR             = f'{DRIVE_PREFIX}/DSF-GNN-2/stock_data'
TECH_FEATURES_PATH   = f'{DRIVE_PREFIX}/tech_features_top50.pt'
PRED_NEXT_CLOSE_CSV  = f'{DRIVE_PREFIX}/pred_next_close.csv'
TICKERS_JSON         = f'{DRIVE_PREFIX}/top50_stocks.json'

START_DATE = '2021-01-01'
END_DATE   = '2025-06-06'

COST_RATE            = 0.0025          # cp = cs = 0.25 %
LOOKBACK_YR          = 252             # 터블런스용
TD3_STEPS            = 200_000

FULL_CAL = pd.date_range(START_DATE, END_DATE, freq='D')

In [27]:
def load_price_data(dir_path: str, tickers: list[str]) -> pd.DataFrame:
    """달력일 전체(full_cal) 기준으로 reindex → Close ffill"""
    frames = []
    for tic in tickers:
        df = pd.read_csv(os.path.join(dir_path, f'{tic}.csv'),
                         parse_dates=['Date'])
        df = df.set_index('Date')[['Close']].sort_index()
        frames.append(df.rename(columns={'Close': tic}))
    price = functools.reduce(lambda a, b: a.join(b, how='outer'), frames)
    # 기간 필터 & 달력일 전체 인덱스
    price = price.sort_index().loc[START_DATE:END_DATE]
    price = price.reindex(FULL_CAL).ffill()
    price.index.name = 'Date'
    return price

def load_tech_features(pt_path: str) -> pd.DataFrame:
    """원본 그대로: DataPreprocessor가 이미 결측 embedding 후 생성한 Tensor"""
    blob   = torch.load(pt_path, map_location='cpu')
    tensor = blob['features'].numpy()      # (T, N, F)
    dates  = pd.to_datetime(blob['meta']['dates'])
    tics   = blob['meta']['symbols']
    cols   = blob['meta']['columns']
    lst = []
    for i, tic in enumerate(tics):
        df = pd.DataFrame(tensor[:, i, :], index=dates, columns=cols)
        df['tic'] = tic
        lst.append(df.set_index('tic', append=True))
    tech = pd.concat(lst)
    tech = tech.sort_index()
    # pt 에 이미 full calendar로 저장됐을 수도, 아니면 아래처럼 강제 reindex+ffill
    tech = tech.unstack('tic')\
               .reindex(FULL_CAL)\
               .ffill()\
               .stack('tic')
    return tech

def build_static_state(price_df: pd.DataFrame,
                       tech_df: pd.DataFrame,
                       pred_csv: str) -> pd.DataFrame:
    # --- 1) Predicted returns ---
    tmp = pd.read_csv(pred_csv)
    if 'Date' in tmp.columns:
        pred = tmp.set_index(pd.to_datetime(tmp['Date'])).iloc[:, 1:]
    else:
        pred = pd.read_csv(pred_csv, index_col=0, parse_dates=True)
    pred.index.name = 'Date'
    pred = pred.loc[START_DATE:END_DATE]
    pred = pred.reindex(FULL_CAL).fillna(method='ffill').fillna(0.0)
    pred_rt = (pred / price_df - 1.0).fillna(0.0)
    pred_rt.columns = [f'pred_rt_{c}' for c in pred_rt.columns]

    # --- 2) Tech features flatten & flatten columns ---
    tech_flat = tech_df.unstack('tic')  # idx=Date, cols=(feature, tic)
    # flatten MultiIndex columns → strings "feature_tic"
    tech_flat.columns = [
        f'{feat}_{tic}'
        for feat, tic in tech_flat.columns
    ]
    tech_flat.index.name = 'Date'
    tech_flat = tech_flat.reindex(FULL_CAL).ffill()

    # --- 3) Concatenate state (no dropna) ---
    state = pd.concat([tech_flat, pred_rt], axis=1)
    # ensure consistent column order
    state = state.sort_index(axis=1)
    return state


In [38]:
# -----------------------------------------------------------------------
# 3. 포트폴리오 환경 (거래비용 μ_t & 이전 비중 state 포함)
# -----------------------------------------------------------------------
class PortfolioEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, price_df: pd.DataFrame, static_state: pd.DataFrame,
                 cost_rate: float = COST_RATE):
        super().__init__()
        self.price_df = price_df.loc[static_state.index]
        self.static_state = static_state
        self.dates = static_state.index
        self.N = price_df.shape[1]                 # 자산 수
        self.cost = cost_rate

        # 액션: 실수 N 벡터 → softmax로 비중 산출
        self.action_space = spaces.Box(-np.inf, np.inf, (self.N,), np.float32)

        # 관측: 정적 feature + 직전 비중(N)
        obs_dim = static_state.shape[1] + self.N
        self.observation_space = spaces.Box(-np.inf, np.inf, (obs_dim,), np.float32)

        self.reset()

    # ---------- 내부 편의 ----------
    def _weights(self) -> np.ndarray:
        # (N,) 자산 비중(현금 제외)
        asset_val = self.holdings * self.cur_prices
        total_val = self.cash + asset_val.sum()
        return (asset_val / total_val).astype(np.float32)

    def _get_obs(self) -> np.ndarray:
        static = self.static_state.iloc[self.idx].values.astype(np.float32)
        return np.concatenate([static, self._weights()])

    # ---------- Gym 필수 ----------
    def reset(self, seed: int | None = None, options=None):
        super().reset(seed=seed)
        self.idx = 0
        self.cur_prices = self.price_df.iloc[self.idx].values
        self.holdings = np.zeros(self.N, dtype=np.float32)
        self.cash = 1_000_000.0
        return self._get_obs(), {}

    def step(self, action: np.ndarray):
        action = np.asarray(action, dtype=np.float32)
        # ----- 1) 직전 비중 (state에 사용, reward 계산용) -----
        w_prev = self._weights()                    # shape (N,)

        # ----- 2) 목표 비중 w_t (softmax) -----
        logits = np.append(action, 0.0)             # 마지막 항: 현금
        w_full = np.exp(logits - logits.max())
        w_full /= w_full.sum()
        w_target_assets = w_full[:-1]               # N

        # ----- 3) 보유 수량 재계산 & 거래비용(μ_t) -----
        port_val = self.cash + (self.holdings * self.cur_prices).sum()
        target_asset_val = w_target_assets * port_val
        delta_val = target_asset_val - self.holdings * self.cur_prices
        turnover = np.sum(np.abs(delta_val)) / port_val
        mu_trade = 1.0 - self.cost * turnover       # μ_t 근사
        # 포트폴리오 가치 감소
        port_val *= mu_trade

        # 주식/현금 업데이트 (비용 차감은 μ_trade에 포함)
        self.holdings = target_asset_val / self.cur_prices
        self.cash = port_val - (self.holdings * self.cur_prices).sum()

        # ----- 4) 다음 시점 이동 -----
        self.idx += 1
        terminated = self.idx >= len(self.dates) - 1
        next_prices = self.price_df.iloc[self.idx].values
        y_t = next_prices / self.cur_prices         # (N,)
        self.cur_prices = next_prices

        # ----- 5) 보상 r_t = ln( μ_t · y_t·w_{t-1} ) -----
        dot = np.dot(y_t, w_prev).item()
        reward = math.log(mu_trade * dot)

        info = {"portfolio_value": port_val * dot}  # 이후 가치
        return self._get_obs(), reward, terminated, False, info

    def render(self):
        print(f"[{self.dates[self.idx].date()}]  PV={self.cash + (self.holdings*self.cur_prices).sum():,.0f}")

In [39]:
# -----------------------------------------------------------------------
# 4. 학습 & 테스트 루틴
# -----------------------------------------------------------------------
def train_td3(env_train, env_eval, steps=TD3_STEPS, save_as='td3_portfolio.zip'):
    n_act = env_train.action_space.shape[-1]
    noise = NormalActionNoise(np.zeros(n_act), 0.1*np.ones(n_act))
    model = TD3("MlpPolicy", env_train, action_noise=noise,
                buffer_size=400_000, batch_size=512, learning_rate=3e-4,
                tau=0.005, gamma=0.99, verbose=1, device='auto')
    model.learn(total_timesteps=steps, log_interval=1000, progress_bar=True)
    model.save(save_as)

    # 백테스트
    obs, _ = env_eval.reset()
    vals = []
    done = False
    while not done:
        act, _ = model.predict(obs, deterministic=True)
        obs, _, done, _, inf = env_eval.step(act)
        vals.append(inf["portfolio_value"])
    returns = np.array(vals) / vals[0]
    print(f"⚑ Cum.Return = {returns[-1]-1:.2%}")
    return model, returns

In [28]:
# -----------------------------------------------------------------------
# 5. main
# -----------------------------------------------------------------------

with open(TICKERS_JSON) as f:
    tickers = json.load(f)['top_stocks']

price = load_price_data(DATA_DIR, tickers)
tech  = load_tech_features(TECH_FEATURES_PATH)
state = build_static_state(price, tech, PRED_NEXT_CLOSE_CSV)

split = state.index[int(len(state)*0.7)]
env_tr = PortfolioEnv(price.loc[:split], state.loc[:split])
env_ev = PortfolioEnv(price.loc[split:], state.loc[split:])

print(price)
print(tech)
print(state)

<ipython-input-27-654ae419b0af>:34: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack('tic')
<ipython-input-27-654ae419b0af>:48: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pred = pred.reindex(FULL_CAL).fillna(method='ffill').fillna(0.0)


               AAPL      ACN    ALGN     AMGN      AXP      BA      BDX  \
Date                                                                      
2021-01-01      NaN      NaN     NaN      NaN      NaN     NaN      NaN   
2021-01-02      NaN      NaN     NaN      NaN      NaN     NaN      NaN   
2021-01-03      NaN      NaN     NaN      NaN      NaN     NaN      NaN   
2021-01-04  126.410  249.916  526.46  213.128  115.187  202.72  243.930   
2021-01-05  127.957  251.344  543.65  214.164  115.813  211.63  244.256   
...             ...      ...     ...      ...      ...     ...      ...   
2025-06-02  201.700  314.470  178.51  288.470  295.330  211.47  169.400   
2025-06-03  203.270  312.900  179.16  289.570  297.390  213.43  170.660   
2025-06-04  202.820  316.400  179.99  287.010  295.930  211.98  170.940   
2025-06-05  200.630  315.380  180.26  287.100  295.960  209.02  171.030   
2025-06-06  204.595  319.530  182.16  291.640  301.460  209.81  173.355   

                CAT     

In [40]:
model, curve = train_td3(env_tr, env_ev,
                            steps=TD3_STEPS,
                            save_as=f'{DRIVE_PREFIX}/td3_portfolio.zip')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: Continuous action space must have a finite lower and upper bound

In [ ]:
plt.plot(state.loc[split:].index, curve)
plt.title("TD3 Portfolio – Backtest"); plt.ylabel("Cumulative Return")
plt.grid(True); plt.show()